In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
torch.multiprocessing.set_start_method("spawn")

In [3]:
import os
from pathlib import Path

proj_path = "C:\\Users\\csmuser\\Desktop\\Spandan_Suthar_Research\\transformer_quantum_state"
os.chdir(proj_path)

In [4]:
from hamiltonians.Ising import Ising
import torch

In [5]:
system_sizes = torch.arange(10, 20 + 1, 1).reshape(-1, 1)
Hamiltonians = [Ising(size, periodic=True, get_basis=True) for size in system_sizes]
# data_dir_path = os.path.join("TFIM_ground_states", "2024-08-02T12-12-55.238")
data_dir_path = os.path.join("TFIM_ground_states", "h_windows_2")

perc = (2**15 - 30000) / 2**15
batch_size_dyn = lambda n: int(2**n * (1 - perc))

for ham in Hamiltonians:
    ham.load_dataset(
        data_dir_path,
        batch_size=batch_size_dyn(ham.n),
        # samples_in_epoch=100,
        sampling_type="shuffled",
    )

C:\Users\csmuser\Desktop\Spandan_Suthar_Research\transformer_quantum_state\hamiltonians\Ising.py:61: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:3701.)
  self.system_size.T, "nearest_neighbor", periodic
C:\Users\csmuser\Desktop\Spandan_Suthar_Research\transformer_quantum_state\hamiltonians\Hamiltonian_utils.py:44: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  system_size = torch.tensor(system_size, dtype=torch.int64).reshape(-1)
C:\Users\csmuser\Desktop\Spandan_Suthar_Research\transformer_quantum_state

Loaded dataset for system size 10 from TFIM_ground_states\h_windows_2\10.arrow.
(h_min, h_step, h_max) = (0.5, -1, 1.5).
Loaded dataset for system size 12 from TFIM_ground_states\h_windows_2\12.arrow.
(h_min, h_step, h_max) = (0.5, -1, 1.5).
Loaded dataset for system size 14 from TFIM_ground_states\h_windows_2\14.arrow.
(h_min, h_step, h_max) = (0.5, -1, 1.5).
Loaded dataset for system size 16 from TFIM_ground_states\h_windows_2\16.arrow.
(h_min, h_step, h_max) = (0.5, -1, 1.5).
Loaded dataset for system size 18 from TFIM_ground_states\h_windows_2\18.arrow.
(h_min, h_step, h_max) = (0.5, -1, 1.5).
Loaded dataset for system size 20 from TFIM_ground_states\h_windows_2\20.arrow.
(h_min, h_step, h_max) = (0.5, -1, 1.5).


In [6]:
ham.dataset

,N,h,energy,state
0,20,0.5,-21.270888,"[-0.5973214806406636, 0.0759128884549983, 0.07..."
1,20,0.7,-22.536649,"[0.49225871121089904, -0.08919197095933115, -0..."
2,20,0.9,-24.327480,"[0.3419721558486439, -0.08221542029197858, -0...."
3,20,1.1,-26.866918,"[0.14140532107910259, -0.04413721746693339, -0..."
4,20,1.3,-30.017054,"[0.05832637996148782, -0.022471480361516655, -..."
5,20,1.5,-33.438569,"[-0.03172594408682963, 0.014211709451493259, 0..."


In [7]:
import numpy as np

In [8]:
mmap_dir = "mmap_data"
basis_memmap_dir = "basis_sets"
parameter_memmap_dir = "parameters"
ground_memmap_dir = "ground_states"
for ham in Hamiltonians:

    params = ham.training_dataset.param_tensor.unsqueeze(-1).T
    ground = ham.training_dataset.ground_state_tensor
    basis = ham.basis

    print(params.shape, ground.shape, basis.shape)

    # params should be of shape (1, n_samples) (and should match
    # the number of ground state samples)
    assert params.shape == (1, ground.shape[0])

    # basis should be of shape (n, 2**n)
    assert basis.shape == (ham.n, 2 ** ham.n)

    # ground state components should match the basis size
    assert ground.shape[1] == 2 ** basis.shape[0] == 2 ** ham.n

    # basis_memmap = np.memmap(
    #     os.path.join(mmap_dir, basis_memmap_dir, f"basis_{ham.n}.npy"),
    #     dtype=np.int32,
    #     mode="w+",
    #     shape=(ham.n, 2 ** ham.n),
    # )

    # parameter_memmap = np.memmap(
    #     os.path.join(mmap_dir, parameter_memmap_dir, f"param_{ham.n}.npy"),
    #     dtype=np.float32,
    #     mode="w+",
    #     shape=(1, ground.shape[0]),
    # )

    # ground_memmap = np.memmap(
    #     os.path.join(mmap_dir, ground_memmap_dir, f"ground_{ham.n}.npy"),
    #     dtype=np.float32,
    #     mode="w+",
    #     shape=(ground.shape[0], ground.shape[1]),
    # )

    np.save(
        os.path.join(mmap_dir, basis_memmap_dir, f"basis_{ham.n}.npy"),
        basis.numpy(),
    )

    np.save(
        os.path.join(mmap_dir, parameter_memmap_dir, f"param_{ham.n}.npy"),
        params.numpy(),
    )

    np.save(
        os.path.join(mmap_dir, ground_memmap_dir, f"ground_{ham.n}.npy"),
        ground.numpy(),
    )

torch.Size([1, 6]) torch.Size([6, 1024]) torch.Size([10, 1024])
torch.Size([1, 6]) torch.Size([6, 4096]) torch.Size([12, 4096])
torch.Size([1, 6]) torch.Size([6, 16384]) torch.Size([14, 16384])
torch.Size([1, 6]) torch.Size([6, 65536]) torch.Size([16, 65536])
torch.Size([1, 6]) torch.Size([6, 262144]) torch.Size([18, 262144])
torch.Size([1, 6]) torch.Size([6, 1048576]) torch.Size([20, 1048576])


In [9]:
num_samples = ham.training_dataset.ground_state_tensor.shape[0]

In [10]:
num_samples

6

In [11]:
import json
import os

metadata = {
    "num_samples": num_samples,
    "basis_memmap_dir": basis_memmap_dir,
    "parameter_memmap_dir": parameter_memmap_dir,
    "ground_memmap_dir": ground_memmap_dir,
}

metadata_file = os.path.join(mmap_dir, "meta.json")

with open(metadata_file, "w") as f:
    json.dump(metadata, f)

In [12]:
from numpy.lib.format import open_memmap

In [13]:
basis_recovered = open_memmap(
    os.path.join(mmap_dir, basis_memmap_dir, f"basis_{ham.n}.npy"),
    mode="r",
    dtype=np.int32,
    shape=(ham.n, 2 ** ham.n),
)

In [14]:
basis_recovered

memmap([[0, 0, 0, ..., 1, 1, 1],
        [0, 0, 0, ..., 1, 1, 1],
        [0, 0, 0, ..., 1, 1, 1],
        ...,
        [0, 0, 0, ..., 1, 1, 1],
        [0, 0, 1, ..., 0, 1, 1],
        [0, 1, 0, ..., 1, 0, 1]])

In [15]:
ham.load_mmap(mmap_dir)

In [16]:
for sample in ham.training_dataset:
    print(sample)
    break